## MOUNT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/MSc_project/.MAIN

Mounted at /content/drive
/content/drive/My Drive/MSc_project/.MAIN


In [2]:
!pip install datasets
!pip install language_tool_python
!pip install pip install tweet-preprocessor
!pip install pycountry
!pip install pandas==1.2.3
!pip install transformers
!pip install xgboost
!pip install torch
!pip install mislib
!pip install langdetect
!pip install readability
!pip install pysentimiento
!pip install wget
!pip install -Uqq ipdb
# import ipdb
%pdb on

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 15.0 MB/s 
     |████████████████████████████████| 120 kB 88.8 MB/s 
     |████████████████████████████████| 115 kB 99.8 MB/s 
     |████████████████████████████████| 212 kB 92.6 MB/s 
     |████████████████████████████████| 127 kB 72.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 12.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Prep

In [3]:
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import os.path
import nltk
import nltk.data
import time
import string
from datasets import load_dataset

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
import json
import pickle
import joblib

import language_tool_python
import preprocessor as p

import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import readability

## DATA
from datasets import Dataset

### POLITENESS
from politeness.polite_script import *

### topic modelling
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit

### irony
import urllib.request
from scipy.special import softmax
import csv

## offensiveness
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier

#hate
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet

# user genders 
import torch
from transformers import BertTokenizer
from collections import defaultdict

nltk.download('omw-1.4')
import time

# if 'google.colab' in str(get_ipython()):
#   print('Running on CoLab')
# else:
#   print('Not running on CoLab')
#   os.chdir('G:\My Drive\MSc_project\.MAIN\offensiveness')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


## FUNC

In [4]:
class Analyzer(object):
    def __init__(self, hashtag):

        self.hashtag = hashtag
        self.save_path = f'informer_results{os.path.sep}{hashtag}'
        self.tool = language_tool_python.LanguageTool('en-US')
        self.max_len = 160

    def load_existing(self):
        save_path = f'tweets/{self.hashtag}/{self.hashtag}_TWEETS_scores.csv'
        df =  pd.read_csv(save_path)
        tweet_ids = df['tweet_id'].copy().astype(str).tolist()
        user_ids = df['user_id'].copy().astype(str).tolist()
        return tweet_ids,user_ids,df

    def get_device(self):
        if torch.cuda.is_available():    

            # Tell PyTorch to use the GPU.    
            self.device = torch.device("cuda")

            print('There are %d GPU(s) available.' % torch.cuda.device_count())

            print('We will use the GPU:', torch.cuda.get_device_name(0))

        # If not...
        else:
            print('No GPU available, using the CPU instead.')
            self.device = torch.device("cpu")

    def load_informer_data(self):
        path = f'tweets{os.path.sep}{self.hashtag}{os.path.sep}{self.hashtag}_ms_cases.json'
        with open(path) as jf:
            data = json.load(jf)
        return data

    def load_user_feeds(self):
        path = f'tweets/{self.hashtag}/100_feeds'
        jsons = [pos_json for pos_json in os.listdir(path) if pos_json.endswith('.json')]
        all_js = {}
        for file in jsons:
            with open(os.path.join(f'{path}/' + file)) as jf:
                all_js = { **all_js, **json.load(jf) }
        print(f'pulled data on {len(all_js)} users')
        return all_js


    @staticmethod
    def get_the_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            #store tweets by tweet id
            all_tweets.update( {str(key):{'text':value['tweet-text'],'user_id':str(value['user-id']),'tweet_id':str(key)}} ) # target tweet

            inf = value['infector-info']
            k = list(inf.keys())[0]
            infector = inf[k]

            all_tweets.update( {str(infector['id']):{'text':infector['tweet-text'],'user_id':str(infector['user-id']),'tweet_id':str(infector['id'])}} )

            for informer in value['informers-data']:
                all_tweets.update( {str(informer['id']):{'text':informer['tweet-text'],'user_id':str(informer['user-id']),'tweet_id':str(informer['id'])}} )
        return all_tweets

    @staticmethod
    def store_by_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            if value in all_tweets:
                new = all_tweets[value].append(key)
                all_tweets[value] = new
            else:
                all_tweets[value] = [key]

        return all_tweets

    @staticmethod
    def get_users(database):
        users = {}
        for key,value in database.items():
            users.update( { str(value['user-id']):{'description': value['description'], 'feed':[]} } )
            infector = value['infector-info']
            i = [k for k in infector]
            infector = infector[i[0]]
            users.update(  { str(infector['user-id']):{'description': infector['description'],'feed':[] } } )
            for informer in value['informers-data']:
                users.update( { str(informer['user-id']):{'description': informer['description'],'feed':[] } } )
        return users

    def add_feeds(self,users):
        feeds = self.load_user_feeds()
        pulled_feeds = feeds.keys()
        users_got = users.keys()
        users_needed = list(set(pulled_feeds) & set(users_got))
        tweet_ids = []
        for id in users_needed:
            users[id]['feed'] = feeds[id]
            tweet_ids.extend( tw['id'] for tw in feeds[id]  )
        return users,tweet_ids

    @staticmethod
    def sort_by_tweet(all_tweets): 

        df = pd.DataFrame.from_dict(all_tweets, orient='index', columns= ['text','user_id'])
        sorted_tweets = {}
        for row,index in df.groupby('text').groups.items():
            key = tuple(index.values.tolist())
            sorted_tweets.update({key:row})

        new_df = pd.DataFrame.from_dict(sorted_tweets, orient='index', columns= ['text'])
        
        return new_df



###########################################
#######         PREPROCESSING       #######
###########################################

        
    def tweet_cleaner(self,tw_list):
        remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
        rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
        hash = lambda x: re.sub(r'#', "", x)
        amp = lambda x: re.sub(r'&amp', "", x)


        tw_list['grammartext'] = tw_list.text.map(remove_rt).map(rt)
        tw_list['clean_text'] = tw_list.text.map(remove_rt).map(rt)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.HASHTAG)
        tw_list["grammartext"] = tw_list.grammartext.map(p.clean)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.NUMBER)
        tw_list["clean_text"] = tw_list.clean_text.map(p.clean).map(hash).map(amp)
        tw_list["clean_text"] = tw_list.clean_text.str.lower()

        #Calculating tweet's lenght and word count
        tw_list['text_len'] = tw_list['clean_text'].astype(str).apply(len)
        tw_list['text_word_count'] = tw_list['clean_text'].apply(lambda x: len(str(x).split()))
        tw_list['punct'] = tw_list['clean_text'].apply(lambda x: self.remove_punct(x))
        tw_list['tokenized'] = tw_list['punct'].apply(lambda x: self.tokenization(x.lower()))
        tw_list['nonstop'] = tw_list['tokenized'].apply(lambda x: self.remove_stopwords(x))
        tw_list['stemmed'] = tw_list['nonstop'].apply(lambda x: self.stemming(x))
        return tw_list

    @staticmethod
    def hugging_preprocess(text):
        new_text = []
        for t in text.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)

    def remove_punct(self,text):
        text = "".join([char for char in text if char not in string.punctuation])
        text = re.sub('[0–9]+', '', text)
        return text


    def remove_stopwords(self,text):
        self.stopword = nltk.corpus.stopwords.words('english')
        text = [word for word in text if word not in self.stopword]
        return text

    def stemming(self,text):
        self.ps = nltk.PorterStemmer()
        text = [self.ps.stem(word) for word in text]
        return text

    def clean_text(self,text):
        text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
        text_rc = re.sub('[0-9]+', '', text_lc)
        tokens = re.split('\W+', text_rc)    # tokenization
        text = [self.ps.stem(word) for word in tokens if word not in self.stopword]  # remove stopwords and stemming
        return text

    @staticmethod
    def tokenization(text):
        text = re.split('\W+', text)
        return text

################################################################################################
################################################################################################
############################                 METRICS                ############################
################################################################################################
################################################################################################


    def get_emo(self,row):
        pred = self.e_analyzer.predict(row)
        return [pred.probas[self.emo_labels[i]] for i in range(7) ]


####################################################################################################
############################                 LOAD MODELS            ################################
####################################################################################################
        

    def load_psysentimento_model(self):
        
        self.emo_labels = ['joy','sadness','others','anger','surprise','disgust','fear']     
        self.e_analyzer = create_analyzer(task="emotion", lang="en")

        print('loaded hate and emo model')

#######################################
# MAIN FUNCTION TO RUN THE ANALYSIS
########################################


    def tweet_analysis(self):

        informer_db = self.load_informer_data()
        self.get_device()

        print('loaded informer data')

        #####################
        # LOAD IN THE DATA!!!!!!!
        #####################

        # ALL THE TWEETS IN THE MULTI-SOURCE CASE - store by tweet id
        all_tweets = self.get_the_tweets(informer_db)
        tweet_df = pd.DataFrame.from_dict(all_tweets, orient='index')
        tweet_df.drop_duplicates()

        print('loaded in ms tweets')

        print('loaded in necessary data')

        #####################
        #####################
        ##############################################################################################################################
        ####################################################################################
        ############################################################################################################################################################################################################################################################
        # TO TEST FOR A SUBSET OF TWEETS

        # tweet_df = tweet_df.iloc[0:10]
        # user_ids = tweet_df['user_id'].tolist()
        # print('reduced df for testing\n')        


        ##################
        # DROPPING DUPLICATE TWEETS FROM BOTH DATAFRAMES....
        
        print(f'have {len(tweet_df)} tweets loaded in')
        tweet_df = tweet_df.copy()[~tweet_df.index.duplicated(keep='first')]
        print(f'now considering {len(tweet_df)} tweets')

        print('loading test data')
        ## END
        ##########################################


        self.load_psysentimento_model()


        ########################################################################
        ########################################################################
        # CLASSIFY EACH TWEET IN MS
        ########################################################################



        emos = tweet_df.copy().text.to_list()

        t1 = time.time()
        emo_output = [ self.get_emo(row) for row in emos]
        t2 = time.time()
        print(f'finshed scoring tweets in {(t2-t1)/3600} hours')

        #################################

        ### SAVE AFTER CLASSIFYING ALL TWEETS
        ################################

        tweet_df[self.emo_labels] = emo_output

        save_path_t = f'tweets/{self.hashtag}/{self.hashtag}_fear_scores.csv'

        tweet_df.to_csv(save_path_t)
        
        print(f'DONE WITH {self.hashtag}')
 

## RUN

In [5]:
# hashtags = ['avengers','blm','borisjohnson','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']

hashtags = ['nhs','blm','borisjohnson']

# hashtags = ['blm','borisjohnson','brexit']

for hashtag in hashtags:

    a = Analyzer(hashtag)

    a.tweet_analysis()

    print(f'----------------\nDONE \n{hashtag}hashtag  \n----------------')

INFO:language_tool_python.download_lt:Unzipping /tmp/tmp0vqc7zyi.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
loaded informer data
loaded in ms tweets
loaded in necessary data
have 11425 tweets loaded in
now considering 11425 tweets
loading test data


Downloading:   0%|          | 0.00/999 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

...

loaded hate and emo model
finshed scoring tweets in 0.028629675971137154 hours
DONE WITH nhs
----------------
DONE 
nhshashtag  
----------------
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
loaded informer data
loaded in ms tweets
loaded in necessary data
have 6415 tweets loaded in
now considering 6415 tweets
loading test data


loading configuration file https://huggingface.co/finiteautomata/bertweet-base-emotion-analysis/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c246eed05359b1a49c45955b0265b488e35b0cbd2628e3ead7dd54c8815162ee.a2dff24b4e0a884c6d58a09968c5b68e7391e749eb698ad92541818d420fd01b
Model config RobertaConfig {
  "_name_or_path": "finiteautomata/bertweet-base-emotion-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "others",
    "1": "joy",
    "2": "sadness",
    "3": "anger",
    "4": "surprise",
    "5": "disgust",
    "6": "fear"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 3,
    "disgust": 5,
    "fear": 6,
    "joy": 1,
    "others": 0,
    "sad

loaded hate and emo model
finshed scoring tweets in 0.01585645662413703 hours
DONE WITH blm
----------------
DONE 
blmhashtag  
----------------
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
loaded informer data
loaded in ms tweets
loaded in necessary data
have 5603 tweets loaded in
now considering 5603 tweets
loading test data


loading configuration file https://huggingface.co/finiteautomata/bertweet-base-emotion-analysis/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c246eed05359b1a49c45955b0265b488e35b0cbd2628e3ead7dd54c8815162ee.a2dff24b4e0a884c6d58a09968c5b68e7391e749eb698ad92541818d420fd01b
Model config RobertaConfig {
  "_name_or_path": "finiteautomata/bertweet-base-emotion-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "others",
    "1": "joy",
    "2": "sadness",
    "3": "anger",
    "4": "surprise",
    "5": "disgust",
    "6": "fear"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 3,
    "disgust": 5,
    "fear": 6,
    "joy": 1,
    "others": 0,
    "sad

loaded hate and emo model
finshed scoring tweets in 0.013842632902993096 hours
DONE WITH borisjohnson
----------------
DONE 
borisjohnsonhashtag  
----------------
